In [1]:
import pandas as pd



In [2]:
data = pd.read_csv('recommendations.csv')

In [4]:
assignees = [
    "All",
    "NIST",
    "Department of Commerce",
    "AISI",
    "Bureau of Industry and Security (BIS)",
    "USPTO",
    "National Semiconductor Technology Center",
    "NOAA",
    "Census Bureau",
    "USTR",
    "NAIC",
    "CHIPS Program Office",
    "Manufacturing USA",
    "PTO",
    "SelectUSA",
    "Patent and Trial Appeal Board",
    "Foreign Commercial Service",
    "NCI",
    "EDA",
    "CHIPS Manufacturing USA",
    "End-User Review Committee",
    "Bureau of Economic Analysis"
]
commerce_recommendations = data[data['Assignee'].isin(assignees)]


In [5]:
commerce_recommendations.to_excel('commerce_recommendations_ai_action_plan_comments.xlsx')

OrgType

In [8]:
import plotly.express as px
import plotly.io as pio
pio.renderers.default = "browser"

# Group and count by OrgType
org_counts = commerce_recommendations['OrgType'].value_counts().reset_index()
org_counts.columns = ['OrgType', 'Count']

# Plot using Plotly Express
fig = px.bar(
    org_counts,
    x='OrgType',
    y='Count',
    title='Count of Recommendations by Organization Type',
    labels={'OrgType': 'Organization Type', 'Count': 'Number of Recommendations'},
    text='Count'
)

fig.update_traces(textposition='outside')
fig.update_layout(xaxis_tickangle=-45)

fig.show()


Assignee

In [9]:

exploded_df = commerce_recommendations.copy()
exploded_df['Topic'] = exploded_df['Topic'].str.split(',')

# Remove leading/trailing whitespace from topics
exploded_df = exploded_df.explode('Topic')
exploded_df['Topic'] = exploded_df['Topic'].str.strip()

# Step 2: Group by cleaned Topic and Assignee, then count
grouped = exploded_df.groupby(['Topic', 'Assignee']).size().reset_index(name='Count')

# Step 3: Pivot the table to get one row per Topic
topic_wide = grouped.pivot(index='Topic', columns='Assignee', values='Count').fillna(0).astype(int).reset_index()

# Optional: sort by total count across all assignees
topic_wide['Total'] = topic_wide.drop(columns='Topic').sum(axis=1)
topic_wide = topic_wide.sort_values(by='Total', ascending=False).drop(columns='Total')

topic_wide


Assignee,Topic,AISI,Bureau of Industry and Security (BIS),CHIPS Program Office,Census Bureau,Department of Commerce,Manufacturing USA,NIST,NOAA,National Semiconductor Technology Center,PTO,Patent and Trial Appeal Board,SelectUSA,USPTO,USTR
16,Standards & Regulation,25,2,0,0,2,1,126,0,2,0,0,0,2,0
5,Evidence of Risks,15,1,0,0,0,0,51,0,0,0,0,0,1,0
15,Security,9,4,0,0,2,0,36,0,1,0,0,0,1,0
10,Infrastructure,2,5,1,0,7,1,10,2,15,0,0,2,1,1
2,Data & IP,0,0,0,2,1,0,14,1,0,1,1,0,17,0
8,Global engagement,6,5,0,0,4,0,21,0,0,0,0,0,0,1
6,Export Controls,0,24,0,0,9,0,0,0,0,0,0,0,1,2
3,Deregulation,2,1,1,0,3,0,14,1,0,0,0,0,6,0
0,Basic Science,2,0,0,0,0,0,6,1,7,0,0,0,1,0
14,Procurement,0,0,0,0,1,0,8,1,0,0,0,2,1,0


In [10]:
df = topic_wide


# Combine AISI and National Semiconductor Technology Center into NIST
df['NIST'] = df['NIST'] + df['AISI'] + df['National Semiconductor Technology Center']

# Combine PTO and Patent and Trial Appeal Board into USPTO
df['USPTO'] = df['USPTO'] + df['PTO'] + df['Patent and Trial Appeal Board']

# Drop merged columns
df = df.drop(columns=['AISI', 'National Semiconductor Technology Center', 'PTO', 'Patent and Trial Appeal Board'])

# Melt to long format
df_long = df.melt(id_vars='Topic', var_name='Assignee', value_name='Count')
df_long = df_long[df_long['Count'] > 0]
# Calculate total counts for each topic
totals = df_long.groupby('Topic')['Count'].sum().reset_index()
# Reorder Topic by total count (largest to smallest)
topic_order = totals.sort_values('Count', ascending=False)['Topic']
df_long['Topic'] = pd.Categorical(df_long['Topic'], categories=topic_order, ordered=True)

# Create stacked bar chart
fig = px.bar(
    df_long,
    x='Topic',
    y='Count',
    color='Assignee',
    title='Recommendation Count by Topic (Stacked by Assignee)',
    labels={'Count': 'Number of Recommendations'},
    color_discrete_sequence=px.colors.qualitative.G10
)

fig.update_layout(barmode='stack', xaxis_tickangle=-45)


# Add annotations
for i, row in totals.iterrows():
    fig.add_annotation(
        x=row['Topic'],
        y=row['Count'],
        text=str(row['Count']),
        showarrow=False,
        yshift=10,
        font=dict(size=12, color='black')
    )

fig.show()


In [11]:
# Reorder Topic by total count (largest to smallest)
topic_order = totals.sort_values('Count', ascending=False)['Topic']
df_long['Topic'] = pd.Categorical(df_long['Topic'], categories=topic_order, ordered=True)
